In [239]:
import torch
import torchvision
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms

In [240]:
df_train = pd.read_csv("/content/drive/MyDrive/Sign_Language/sign_mnist_train.csv")
df_test = pd.read_csv("/content/drive/MyDrive/Sign_Language/sign_mnist_test.csv")

In [241]:
# Separate labels and pixel values
labels_train = df_train.iloc[:, 0].values
pixels_train = df_train.iloc[:, 1:].values
print(f"labels = {labels_train}\npixels = {pixels_train}")

labels = [ 3  6  2 ... 18 17 23]
pixels = [[107 118 127 ... 204 203 202]
 [155 157 156 ... 103 135 149]
 [187 188 188 ... 195 194 195]
 ...
 [174 174 174 ... 202 200 200]
 [177 181 184 ...  64  87  93]
 [179 180 180 ... 205 209 215]]


In [242]:
# Separate labels and pixel values
labels_test = df_test.iloc[:, 0].values
pixels_test = df_test.iloc[:, 1:].values
print(f"labels = {labels_test}\npixels = {pixels_test}")

labels = [ 6  5 10 ...  2  4  2]
pixels = [[149 149 150 ... 112 120 107]
 [126 128 131 ... 184 182 180]
 [ 85  88  92 ... 225 224 222]
 ...
 [190 191 190 ... 211 209 208]
 [201 205 208 ...  67  70  63]
 [173 174 173 ... 195 193 192]]


In [243]:
class AlexNet(nn.Module):
  def __init__(self, num_classes, dropout=0.5):
      super(AlexNet, self).__init__()
      self.features = nn.Sequential(
          nn.Conv2d(1, 64, 5, stride=1, padding=2),  # Adjusted kernel size, stride, and padding
          nn.ReLU(inplace=True),
          nn.MaxPool2d(kernel_size=2, stride=2),  # Adjusted kernel size and stride
          nn.Conv2d(64, 192, 5, stride=1, padding=2),
          nn.ReLU(inplace=True),
          nn.MaxPool2d(kernel_size=2, stride=2),
          nn.Conv2d(192, 384, 3, stride=1, padding=1),
          nn.ReLU(inplace=True),
          nn.Conv2d(384, 256, 3, stride=1, padding=1),
          nn.ReLU(inplace=True),
          nn.Conv2d(256, 256, 3, stride=1, padding=1),
          nn.ReLU(inplace=True),
          nn.MaxPool2d(kernel_size=2, stride=2)  # Adjusted kernel size and stride
      )
      self.classifier = nn.Sequential(
          nn.Dropout(p=dropout),
          nn.Linear(256 * 3 * 3, 4096),  # Adjusted dimensions based on feature map size
          nn.ReLU(inplace=True),
          nn.Dropout(p=dropout),
          nn.Linear(4096, 4096),
          nn.ReLU(inplace=True),
          nn.Linear(4096, num_classes),  # Adjusted to match the number of classes
          nn.Softmax(dim=1)
      )

  def forward(self, x):
      x = self.features(x)
      x = torch.flatten(x, start_dim=1)  # Flatten the tensor
      x = self.classifier(x)
      return x

In [244]:
NUM_CLASSES = df_train.iloc[:, 0].nunique()
print(f'Number of unique labels: {NUM_CLASSES}')

Number of unique labels: 24


In [245]:
# Define your device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = AlexNet(num_classes=NUM_CLASSES).to(device)
epochs = 20 # number of single passes on the network
loss_fnPrecents = nn.CrossEntropyLoss(ignore_index=24)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)

In [246]:
class SignDataset(Dataset):
    def __init__(self, pixels, labels, transform=None):
        self.pixels = pixels
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        image = self.pixels[idx].reshape(28, 28).astype('float32')
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

In [250]:
transform = transforms.Compose([
    transforms.ToTensor()
])
train_dataset = SignDataset(pixels_train, labels_train, transform=transform)
train_dataloader = DataLoader(train_dataset, batch_size=200)

In [251]:
# Training loop
def train_model(model, dataloader, epochs, optimizer, loss_fn, device):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)

            # Forward pass
            outputs = model.forward(images)
            loss = loss_fn(outputs, labels)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(dataloader):.4f}")

def evaluate_model(model, dataloader, device):
    model.eval()  # Set the model to evaluation mode
    total, correct = 0, 0
    with torch.no_grad():
        for data, targets in dataloader:
            data, targets = data.to(device), targets.to(device)  # Ensure data and targets are on the correct device
            outputs = model.forward(data)
            _, predicted = torch.max(outputs.data, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()

    accuracy = 100 * correct / total  # Calculate accuracy as a percentage
    print(f"Total Correct: {correct}")
    print(f"Total Samples: {total}")
    print(f"Accuracy: {accuracy}%")

In [253]:
train_model(model=model,
            dataloader=train_dataloader,
            epochs=epochs,
            optimizer=optimizer,
            loss_fn=loss_fnPrecents,
            device=device)

Epoch [1/20], Loss: 3.2013
Epoch [2/20], Loss: 3.2013
Epoch [3/20], Loss: 3.2013
Epoch [4/20], Loss: 3.2013
Epoch [5/20], Loss: 3.2013
Epoch [6/20], Loss: 3.2013
Epoch [7/20], Loss: 3.2013
Epoch [8/20], Loss: 3.2013
Epoch [9/20], Loss: 3.2013
Epoch [10/20], Loss: 3.2013
Epoch [11/20], Loss: 3.2013
Epoch [12/20], Loss: 3.2013
Epoch [13/20], Loss: 3.2013
Epoch [14/20], Loss: 3.2013
Epoch [15/20], Loss: 3.2013
Epoch [16/20], Loss: 3.2013
Epoch [17/20], Loss: 3.2013
Epoch [18/20], Loss: 3.2013
Epoch [19/20], Loss: 3.2013
Epoch [20/20], Loss: 3.2013


In [254]:
val_dataset = SignDataset(pixels_test, labels_test, transform=transform)
val_dataloader = DataLoader(val_dataset, batch_size=200)

In [255]:
evaluate_model(
    model=model,
    dataloader=train_dataloader,
    device=device
)

Total Correct: 1199
Total Samples: 27455
Accuracy: 4.3671462393006735%


In [256]:
evaluate_model(
    model=model,
    dataloader=val_dataloader,
    device=device
)

Total Correct: 246
Total Samples: 7172
Accuracy: 3.4300055772448412%
